In [0]:
import numpy as np
import keras
import keras.utils
from keras import backend as K
from keras import Model
from keras import utils as np_utils
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.layers import Softmax, Dropout,Activation
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers.pooling import GlobalAveragePooling2D


In [0]:
from google.colab import drive

In [34]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [36]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
         )

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/damage_dataset/train',  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 256x256
        batch_size=batch_size,
        class_mode='categorical')  # 
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/damage_dataset/valid',
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical')
validation_generator.target_size


Found 1860 images belonging to 8 classes.
Found 396 images belonging to 8 classes.


(256, 256)

In [0]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("withoutaug.h5", monitor='val_acc', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=15, verbose=1, mode='auto')

In [0]:
from keras import applications
epochs = 50

base_model =applications.inception_v3.InceptionV3(include_top=False, weights='imagenet',
                               input_tensor=None, input_shape=None, pooling=None, classes=8)


CLASSES = 8
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)


for layer in base_model.layers:
    layer.trainable = False 

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    




In [41]:
# Train the model 
history = model.fit_generator(
train_generator,
epochs = epochs,
steps_per_epoch = 60 ,
validation_data = validation_generator,
validation_steps = 16,
callbacks = [checkpoint, early])

Epoch 1/50
60/60 [==============================] - 62s 1s/step - loss: 0.4347 - acc: 0.8636 - val_loss: 2.1349 - val_acc: 0.6589

Epoch 00001: val_acc did not improve from 0.68496
Epoch 2/50
60/60 [==============================] - 63s 1s/step - loss: 0.4684 - acc: 0.8481 - val_loss: 2.3088 - val_acc: 0.6667

Epoch 00002: val_acc did not improve from 0.68496
Epoch 3/50
60/60 [==============================] - 60s 1s/step - loss: 0.4646 - acc: 0.8480 - val_loss: 2.5536 - val_acc: 0.6341

Epoch 00003: val_acc did not improve from 0.68496
Epoch 4/50
60/60 [==============================] - 61s 1s/step - loss: 0.4373 - acc: 0.8583 - val_loss: 2.4509 - val_acc: 0.6260

Epoch 00004: val_acc did not improve from 0.68496
Epoch 5/50
60/60 [==============================] - 61s 1s/step - loss: 0.4596 - acc: 0.8479 - val_loss: 2.7567 - val_acc: 0.6186

Epoch 00005: val_acc did not improve from 0.68496
Epoch 6/50
60/60 [==============================] - 60s 997ms/step - loss: 0.4554 - acc: 0.8521